In [6]:
import pandas as pd
import os

lista_personaggi = ['Ariel_Sharon', 'Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Hugo_Chavez', 'Tony_Blair']

folder_path = 'C:/Users/sollillo1/pythonProjects/fondamenti/Progetto_9/lfw_funneled/'

#definiamo una funzione per creare un DataFrame che fornisca dati sul sottoinsieme di interesse
def filter(file):
    file_name = os.path.join(folder_path, file)
    data = []

    try:
        with open(file_name, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        personaggio, image_name = line.split('/')
                        if personaggio in lista_personaggi:
                            data.append([personaggio, image_name])
                    except ValueError:
                        print(f"Linea malformata in {file}:{line}")
    except FileNotFoundError:
        print(f"File non trovato: {file_name}")
    
    return pd.DataFrame(data, columns = ['Personaggio', 'Nome_immagine'])

#Consideriamo solo i file .txt che contengono informazioni sui personaggi del nostro sottoinsieme
files = ['pairs_01.txt', 'pairs_02.txt', 'pairs_03.txt', 'pairs_04.txt','pairs_05.txt','pairs_06.txt', 'pairs_07.txt','pairs_08.txt', 'pairs_09.txt', 'pairs_10.txt']

#Inseriamo i dati rilevanti ricavati da ciascun file in un unico dataframe
df_unico = pd.concat([filter(file) for file in files], ignore_index = True)

#Eliminiamo elementi duplicati 
df_unico.drop_duplicates(['Personaggio', 'Nome_immagine'], inplace=True)
display(df_unico)

foto_pers = df_unico['Personaggio'].value_counts()

,Personaggio,Nome_immagine
0,Hugo_Chavez,Hugo_Chavez_0002.jpg
1,Hugo_Chavez,Hugo_Chavez_0027.jpg
3,Hugo_Chavez,Hugo_Chavez_0053.jpg
4,Hugo_Chavez,Hugo_Chavez_0036.jpg
5,Hugo_Chavez,Hugo_Chavez_0051.jpg
...,...,...
60,George_W_Bush,George_W_Bush_0145.jpg
61,George_W_Bush,George_W_Bush_0150.jpg
63,George_W_Bush,George_W_Bush_0238.jpg
64,George_W_Bush,George_W_Bush_0203.jpg


In [7]:
#ristrutturo i dati in due DataFrame
pairs = pd.DataFrame(columns=['Personaggio','Foto_idx1', 'Foto_idx2'])
pairs2 = pd.DataFrame(columns=['Personaggio1', 'Foto1', 'Personaggio2', 'Foto2'])

def formatta_dati(line):
    global pairs
    global pairs2

    line = line.strip()
    elem = line.split()
    
    if len(elem) == 3:
        personaggio, n_foto1, n_foto2 = elem
        if personaggio in lista_personaggi:
            new_row = pd.DataFrame( {'Personaggio': [personaggio], 'Foto_idx1': [n_foto1], 'Foto_idx2':[n_foto2]} )
            pairs = pd.concat([pairs, new_row], ignore_index = True)
    elif len(elem) == 4:
        personaggio1, n_foto1, personaggio2, n_foto2 = elem
        if personaggio1 in lista_personaggi or personaggio2 in lista_personaggi:
            new_row =  pd.DataFrame({'Personaggio1': [personaggio1], 'Foto1': [n_foto1], 'Personaggio2':[personaggio2], 'Foto2':[n_foto2]})
            pairs2 = pd.concat([pairs2,new_row], ignore_index = True)

        


pairs_file = os.path.join(folder_path, 'pairs.txt')
with open(pairs_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        formatta_dati(line)

print("\nCoppie di immagini dello stesso personaggio:")
display(pairs)
print("\nCoppie di immagini di personaggi diversi:")
display(pairs2)


Coppie di immagini dello stesso personaggio:


,Personaggio,Foto_idx1,Foto_idx2
0,Hugo_Chavez,2,27
1,Hugo_Chavez,27,53
2,Hugo_Chavez,36,51
3,Hugo_Chavez,41,50
4,Hugo_Chavez,45,56
5,Tony_Blair,18,86
6,Tony_Blair,32,48
7,Tony_Blair,32,110
8,Tony_Blair,53,102
9,Tony_Blair,91,134



Coppie di immagini di personaggi diversi:


,Personaggio1,Foto1,Personaggio2,Foto2
0,Hugo_Chavez,33,Karen_Lynn_Gorney,1
1,Hugo_Chavez,60,Steve_Shiver,1
2,Takahiro_Mori,1,Tony_Blair,64
3,Colin_Powell,95,Frank_Hsieh,1
4,Ain_Seppik,1,Gerhard_Schroeder,55
5,Ariel_Sharon,30,David_Ballantyne,1


In [8]:
import numpy as np                
from matplotlib.image import imread 
import matplotlib.pyplot as plt
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_crop_face(gray_img):
    #rilevo i volti (l'immagine è già in scala di grigi)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) == 0:
        #se nessun volto è rilevato, ritorna None
        return None
    
    #Prendo il primo volto rilevato
    (x, y, w, h) = faces[0]
    cropped_face = gray_img[y:y+h, x:x+w]
    
    return cropped_face

subset_foto = []
#etichette per addestrare il modello 
personaggi_labels = []
nfaces_dict = {}

for personaggio in lista_personaggi:
    nfaces_dict[personaggio] = 0

for nome_cartella in os.listdir(folder_path): 
    percorso_cartella = os.path.join(folder_path, nome_cartella) #creiamo  il percorso completo alla cartella

    #ci accertiamo di accedere solo alle cartelle dei personaggi di interesse
    if os.path.isdir(percorso_cartella) and nome_cartella in lista_personaggi:
        #iteriamo attraverso le foto della cartella
        for file_name in os.listdir(percorso_cartella):
            file_path = os.path.join(percorso_cartella, file_name)
            #verifichiamo che il file a cui cerchiamo di accedere sia un'immagine e che non sia un duplicato
            if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                #utilizziamo imread per leggere l'immagine e caricarla in un array
                img = imread(file_path)
                #riduciamo le immagini a colori in scala di grigi calcolando la media dei valori della dimensione contenete i canali di colore
                #   print("Dimensione immagine a colori:", img.shape)
                img = np.mean(img , -1).astype(np.uint8)
                #   print("Dimensione immagine in scala di grigi:", img.shape)
                
                cropped_face = detect_and_crop_face(img)
                if cropped_face is not None:
                    #le immagini ritagliate possono avere diverse dimensoni 
                    resized_face =cv2.resize(cropped_face, (250,250))
                    #trasformiamo l'array multidimensionale dell'immagine appena aperta in un array unidimensionale
                    flattened_img = resized_face.flatten()
                    #   print("Dimensione appiattita:", flattened_img.shape)
                    subset_foto.append(flattened_img) 
                    personaggi_labels.append(nome_cartella)
                    nfaces_dict[nome_cartella] +=1


nfaces = []           
for personaggio in lista_personaggi:
    nfaces.append(nfaces_dict.get(personaggio, 0))

print("nfaces:", nfaces)
  
# X è la matrice di tutte le immagini appiattite, (250*250, N_immagini), ogni colonna rappresenta un'immagine
X = np.array(subset_foto).T
print(f"Dimensioni matrice di tutte le foto: {X.shape}")

nfaces: [77, 235, 121, 529, 109, 71, 144]
Dimensioni matrice di tutte le foto: (62500, 1286)


In [9]:
#normalizzazione dei dati
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)